## Loading the libraries

In [ ]:
library(tidyverse)
library(reshape2)
library(here)
library(readxl)
#library(limma)
library(multcompView)

In [ ]:
data <- read_excel(here("data", "Phenotype_data", "Screeningdaten.xlsx"))
sampleNames <- read_excel(here("data", "Phenotype_data", "22-01-2021_Zuordnung_genotypes_SB.aktualisiert.xlsx"))

In [ ]:
# Only taking the data for which we have unique identifier of genotyped samples in the 'sampleNames'
data <- data[1:191,]
sampleNames <- sampleNames[1:189,c(1,5)]

In [ ]:
# Deleting rows from the data with unmeasured phenotype, for matching the sample identifier
data <- data[-c(139,156), ]

In [ ]:
dim(data)
dim(sampleNames)

In [ ]:
# Specifying different data frames to concatenate when desired
commonData <- data[1:12] # Data common to all measured traits
dataStarch <- data[53:ncol(data)] # Data concerning the starch content at day 14
# new column names for the data frame dataStarch
newColNamesStarch <- c('CR1', 'CR2', 'CR3', 'CR4', 'MW_C', 'SD_C', 'HR1', 'HR2', 'HR3', 'HR4', 'MW_H', 'SD_H', 'ttest_HC', 'HdC_perc', 'Trait')
colnames(dataStarch) <- newColNamesStarch

In [ ]:
# Appending the common data by the identifier for the GWAS
commonData <- merge(commonData, sampleNames, by = "Nr")

In [ ]:
commonData

In [ ]:
# Separating the starch data into heat and control and calculate average of all observed data points for each category
dataStarchControl <- dataStarch %>%
    select(CR1, CR2, CR3, CR4)
dataStarchControl <- dataStarchControl %>% 
    mutate(CR = rowMeans(select(., starts_with("CR")), na.rm = TRUE))

dataStarchHeat <- dataStarch %>%
    select(HR1, HR2, HR3, HR4)
dataStarchHeat <- dataStarchHeat %>% 
    mutate(HR = rowMeans(select(., starts_with("HR")), na.rm = TRUE))

In [ ]:
dim(dataStarchHeat)
dim(dataStarchControl)
dim(commonData)

In [ ]:
# Creating a new data frame containing only the necessary columns for the starch content
dataStarchNew <- cbind(commonData, dataStarchHeat, dataStarchControl)
# Creating percentage ratio of starch concentration for Heat/Control
dataStarchNew$Perc <- 100 * (dataStarchNew$HR / dataStarchNew$CR)

In [ ]:
dataStarchNew

In [ ]:
# Selecting only experiments 1 to 4 for GWAS
expStarchGwas <- dataStarchNew %>%
    filter(Exp. == "Exp_01" | Exp. == "Exp_02" | Exp. == "Exp_03" | Exp. == "Exp_04")
# Removing rows with NAs in the 'Perc" column'
expStarchGwas <- subset(expStarchGwas, (!is.na(expStarchGwas$Perc)))

In [ ]:
nrow(expStarchGwas)

In [ ]:
#########################
#                       #
# Tukey HSD for Control #
#                       #
#########################

# Using Tukeys HSD to check for significant differences of means between experiments

# First, creating linear model of HR vs. Experiment
modelC <- lm(expStarchGwas$CR ~ expStarchGwas$Exp.)
ANOVAC <- aov(modelC)

# Tukey test to test each pair of experiments
TUKEYC <- TukeyHSD(x=ANOVAC, 'expStarchGwas$Exp.', conf.level = 0.95)

# Plotting the test result
par(mar=c(5.1, 8.1, 4.1, 2.1))
plot(TUKEYC, las = 1, col = 'brown')


######################
#                    #
# Tukey HSD for Heat #
#                    #
######################

# Using Tukeys HSD to check for significant differences of means between experiments

# First, creating linear model of HR vs. Experiment
modelH <- lm(expStarchGwas$HR ~ expStarchGwas$Exp.)
ANOVAH <- aov(modelH)

# Tukey test to test each pair of experiments
TUKEYH <- TukeyHSD(x=ANOVAH, 'expStarchGwas$Exp.', conf.level = 0.95)

# Plotting the test result
par(mar=c(5.1, 8.1, 4.1, 2.1))
plot(TUKEYH, las = 1, col = 'brown')

###################################################
#                                                 #
# Tukey HSD for the percent ratio of Heat/Control #
#                                                 #
###################################################

# Using Tukeys HSD to check for significant differences of means between experiments

# First, creating linear model of HR vs. Experiment
modelP <- lm(expStarchGwas$Perc ~ expStarchGwas$Exp.)
ANOVAP <- aov(modelP)

# Tukey test to test each pair of experiments
TUKEYP <- TukeyHSD(x=ANOVAP, 'expStarchGwas$Exp.', conf.level = 0.95)

# Plotting the test result
par(mar=c(5.1, 8.1, 4.1, 2.1))
plot(TUKEYP, las = 1, col = 'brown')

In [ ]:
summary(ANOVAP)

In [ ]:
# Defining a customized tukey plot layout
tuk_plot <- function (x, xlab, ylab, ylabels = NULL, ...) {
  for (i in seq_along(x)) {
    xi <- x[[i]][, -4L, drop = FALSE]
    yvals <- nrow(xi):1L
    dev.hold()
    on.exit(dev.flush())
    plot(c(xi[, "lwr"], xi[, "upr"]), rep.int(yvals, 2L), 
         type = "n", axes = FALSE, xlab = "", ylab = "", main = NULL, 
         ...)
    axis(1, ...)
    # change for custom axis labels
    if (is.null(ylabels)) ylabels <- NULL # dimnames(xi)[[1L]]

    axis(2, at = nrow(xi):1, labels = ylabels, 
         srt = 180, ...)
    abline(h = yvals, lty = 1, lwd = 0.5, col = "lightgray")
    abline(v = 0, lty = 2, lwd = 0.5, ...)
    segments(xi[, "lwr"], yvals, xi[, "upr"], yvals, ...)
    segments(as.vector(xi), rep.int(yvals - 0.1, 3L), as.vector(xi), 
             rep.int(yvals + 0.1, 3L), ...)
    title(main = paste0(format(100 * attr(x, ""), 
                               digits = 2L), ""), #"% family-wise confidence level\n"
          # change for custom axis titles
          xlab = xlab, ylab = ylab)

    box()
    dev.flush()
    on.exit()
  }
}

In [ ]:
# Plotting the test result
par(mar=c(5.1, 8.1, 4.1, 2.1))
plot(TUKEYP, las = 1, col = 'brown')

In [ ]:
#TUKEYC
#TUKEYH
TUKEYP

In [ ]:
#ggplot(data = expStarchGwas, aes(x=Exp., y=CR)) + 
#    geom_boxplot()
#ggplot(data = expStarchGwas, aes(x=Exp., y=HR)) + 
#    geom_boxplot()
ggplot(data = expStarchGwas, aes(x=Exp., y=Perc)) + 
    geom_boxplot()

## Producing batch analysis plots for MA

In [ ]:
head(expStarchGwas)

In [ ]:
#png(filename=here('Figures', 'Phenotypic_data', 'pheno_batch_effect.png'), res=150, width = 1500, height = 1000)
# Layout to split the screen
layout(mat = matrix(c(1,2),1,2, byrow=TRUE))
# Draw the boxplot and the tukey plot
# First, the boxplot
par(mar=c(5.1, 5, 4.1, 2.1))
boxplot(Perc ~ Exp., data = expStarchGwas, xaxt = 'n', yaxt = 'n', xlab = '', ylab = '')
axis(side = 1, las = 1)
axis(side = 2, las = 2, mpg = c(3,1,0), at = seq(0,180,20), labels = seq(0,180,20))
mtext(side=1, line=3.2, "Batch", cex=1.4)
mtext(side=2, line=3, "Starch concentration ratio [%]", cex=1.4)
mtext("A", adj=-0.25, line=1, font = 2, cex = 1.7)
# Second, the tukey plot
par(mar=c(5.1, 5, 4.1, 2.1))
tuk_plot(TUKEYP, '', '', xaxt = 'n', yaxt = 'n')
mtext(side=1, line=3.2, "Differences in means", cex=1.4)
mtext(side=2, line=3, "Pairwise comparisons between batches", cex=1.4)
axis(side = 1, las = 1, at = seq(-50, 20, 10), labels = seq(-50, 20, 10))
axis(side = 2, las = 2, mpg = c(3,1,0), at = c(1,2,3,4,5,6), labels = c("3-4", "2-4", "2-3", "1-4", "1-3", "1-2"))
mtext("B", adj=-0.25, line=1, font = 2, cex = 1.7)
#dev.off()

In [ ]:
#png(filename=here('Figures', 'Phenotypic_data', 'pheno_histogram.png'), res=100, width = 1000, height = 1000)
# Layout to split the screen
layout(mat = matrix(c(1,2),2,1, byrow=TRUE),  height = c(6,1))
# Draw the boxplot and the histogram 
par(mar=c(4.1, 4.3, 3, 2.1))
hist(phenoGWAS$Trait, breaks=20, col=rgb(0.2,0.8,0.5,0.5), border=T, main="", xlab="", ylab = "", xlim=c(-10,180),
     ylim=c(0, 40), xaxt = 'n', , yaxt = 'n')
#title("Distribution of starch ratio between heat-treated\nand control measurements")
axis(side=1, at=seq(0,180,20), labels=seq(0,180,20), cex.axis=1.3)
axis(side=2, at=seq(0,40,10), labels=seq(0,40,10), cex.axis=1.5)
mtext(side=1, line=4.3, "Starch concentration ratio\n(Heat/Control) [%]", cex=1.4)
mtext(side=2, line=3, "Frequency", cex=1.4)
par(mar=c(0, 4.3, 2, 2.1))
boxplot(phenoGWAS$Trait, horizontal=TRUE , ylim=c(-10,180), xaxt="n", col=rgb(0.2,0.8,0.5,0.5), frame=F)
#dev.off()

In [ ]:
phenoGWAS <- expStarchGwas %>%
    select(Identifier, Perc)
colnames(phenoGWAS) <- c("Individual", "Trait")

In [ ]:
# Adopting individual names to fit to the VCF files
phenoGWAS$Individual <- paste0("Sample_", phenoGWAS$Individual)

In [ ]:
phenoGWAS$Trait <- round(phenoGWAS$Trait,2)

In [ ]:
sd(phenoGWAS$Trait)

In [ ]:
mean(phenoGWAS$Trait)
mean(phenoGWAS$Trait) + 2 * sd(phenoGWAS$Trait)
mean(phenoGWAS$Trait) - 2 * sd(phenoGWAS$Trait)

In [ ]:
phenoGWAS$Individual[!(phenoGWAS$Trait <= mean(phenoGWAS$Trait) + 2 * sd(phenoGWAS$Trait) & phenoGWAS$Trait >= mean(phenoGWAS$Trait) - 2 * sd(phenoGWAS$Trait))]
phenoGWAS$Trait[!(phenoGWAS$Trait <= mean(phenoGWAS$Trait) + 2 * sd(phenoGWAS$Trait) & phenoGWAS$Trait >= mean(phenoGWAS$Trait) - 2 * sd(phenoGWAS$Trait))]

In [ ]:
phenoGWAS$Trait <= mean(phenoGWAS$Trait) + 2 * sd(phenoGWAS$Trait) & phenoGWAS$Trait >= mean(phenoGWAS$Trait) - 2 * sd(phenoGWAS$Trait)

In [ ]:
# Layout to split the screen
layout(mat = matrix(c(1,2),2,1, byrow=TRUE),  height = c(6,1))
# Draw the boxplot and the histogram 
par(mar=c(4.1, 4.3, 3, 2.1))
hist(phenoGWAS$Trait, breaks=20, col=rgb(0.2,0.8,0.5,0.5), border=T, main="", xlab="", ylab = "", xlim=c(-10,180),
     ylim=c(0, 40), xaxt = 'n', , yaxt = 'n')
title("Distribution of starch ratio between heat-treated\nand control measurements")
axis(side=1, at=seq(0,180,20), labels=seq(0,180,20), cex.axis=1.3)
axis(side=2, at=seq(0,40,10), labels=seq(0,40,10), cex.axis=1.5, las =2)
mtext(side=1, line=4.3, "Starch concentration ratio\n(Heat/Control) [%]", cex=1.4)
mtext(side=2, line=3, "Frequency", cex=1.4)
par(mar=c(0, 4.3, 2, 2.1))
boxplot(phenoGWAS$Trait, horizontal=TRUE , ylim=c(-10,180), xaxt="n", col=rgb(0.2,0.8,0.5,0.5), frame=F)

In [ ]:
png(filename=here('Figures', 'Phenotypic_data', 'pheno_histogram.png'), res=100, width = 1000, height = 1000)
# Layout to split the screen
layout(mat = matrix(c(1,2),2,1, byrow=TRUE),  height = c(6,1))
# Draw the boxplot and the histogram 
par(mar=c(4.1, 4.3, 3, 2.1))
hist(phenoGWAS$Trait, breaks=20, col=rgb(0.2,0.8,0.5,0.5), border=T, main="", xlab="", ylab = "", xlim=c(-10,180),
     ylim=c(0, 40), xaxt = 'n', , yaxt = 'n')
#title("Distribution of starch ratio between heat-treated\nand control measurements")
axis(side=1, at=seq(0,180,20), labels=seq(0,180,20), cex.axis=1.3)
axis(side=2, at=seq(0,40,10), labels=seq(0,40,10), cex.axis=1.5)
mtext(side=1, line=4.3, "Starch concentration ratio\n(Heat/Control) [%]", cex=1.4)
mtext(side=2, line=3, "Frequency", cex=1.4)
par(mar=c(0, 4.3, 2, 2.1))
boxplot(phenoGWAS$Trait, horizontal=TRUE , ylim=c(-10,180), xaxt="n", col=rgb(0.2,0.8,0.5,0.5), frame=F)
dev.off()

In [ ]:
png(filename=here('Figures', 'Phenotypic_data', 'pheno_histogram_qq-plot.png'), res=150, width = 1500, height = 1000)
layout(matrix(c(1,2,
                3,3), nrow = 2, byrow = FALSE),
       height = c(3,1),
       width = c(1,1))

par(mar=c(3.5, 4.7, 3, 2.1))
hist(phenoGWAS$Trait, breaks=20, col=rgb(0.2,0.8,0.5,0.5), border=T, main="", xlab="", ylab = "", xlim=c(-10,180),
     ylim=c(0, 40), xaxt = 'n', , yaxt = 'n')
#title("Distribution of starch ratio between heat-treated\nand control measurements")
axis(side=1, at=seq(0,180,30), labels=seq(0,180,30), cex.axis=1.3)
axis(side=2, las=2, at=seq(0,40,10), labels=seq(0,40,10), cex.axis=1.3)
mtext(side=1, line=5.2, "Starch concentration ratio\n(Heat/Control) [%]", cex=1.4)
mtext(side=2, line=3, "Frequency", cex=1.4)
mtext("A", adj=-0.2, line=1, font = 2, cex = 1.7)

par(mar=c(0, 4.7, 2, 2.1))
boxplot(phenoGWAS$Trait, horizontal=TRUE , ylim=c(-10,180), xaxt="n", col=rgb(0.2,0.8,0.5,0.5), frame=F)

par(mar=c(8, 4.7, 3, 2.1))
qqnorm(phenoGWAS$Trait, main="", xlab="", ylab = "", , xaxt = 'n', yaxt = 'n')
axis(side=1, at=seq(-3,3,1), labels=seq(-3,3,1), cex.axis=1.3)
axis(side=2, las =2, at=seq(0,180,20), labels=seq(0,180,20), cex.axis=1.3)
mtext(side=1, line=4.3, "Theoretical quantiles", cex=1.4)
mtext(side=2, line=3.3, "Sample quantiles", cex=1.4)
mtext("B", adj=-0.21, line=1, font = 2, cex = 1.7)
qqline(phenoGWAS$Trait, datax = FALSE, distribution = qnorm, qtype = 7, col = 'red')
#layout.show(n=3)
dev.off()

In [ ]:
qqnorm(phenoGWAS$Trait)
qqline(phenoGWAS$Trait, datax = FALSE, distribution = qnorm, qtype = 7, col = 'red')

In [ ]:
CDF <- ecdf(phenoGWAS$Trait)

In [ ]:
shapiro.test(phenoGWAS$Trait)

In [ ]:
dev.off()

In [ ]:
outPath <- here("data", "Phenotype_data", "GWAS_input.csv")

In [ ]:
head(phenoGWAS)

In [ ]:
write.csv(phenoGWAS, outPath, row.names=FALSE, quote=FALSE)